In [80]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from datetime import datetime
import psycopg2

In [81]:
load_dotenv()

True

### Connect to API and get data

In [82]:
api_key = os.getenv("API_KEY")

In [122]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey={api_key}'

r = requests.get(url)
status_code = r.status_code
print(status_code)
# HTTP response status codes=  https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status

200


In [123]:
data = r.json()
data

{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency',
  '2. Digital Currency Code': 'BTC',
  '3. Digital Currency Name': 'Bitcoin',
  '4. Market Code': 'EUR',
  '5. Market Name': 'Euro',
  '6. Last Refreshed': '2025-04-04 00:00:00',
  '7. Time Zone': 'UTC'},
 'Time Series (Digital Currency Daily)': {'2025-04-04': {'1. open': '75324.46000000',
   '2. high': '75535.33000000',
   '3. low': '75075.93000000',
   '4. close': '75099.15000000',
   '5. volume': '7.39032644'},
  '2025-04-03': {'1. open': '75661.51000000',
   '2. high': '76895.06000000',
   '3. low': '73069.00000000',
   '4. close': '75302.11000000',
   '5. volume': '710.68541996'},
  '2025-04-02': {'1. open': '78900.68000000',
   '2. high': '81300.00000000',
   '3. low': '75600.00000000',
   '4. close': '75617.59000000',
   '5. volume': '685.40827378'},
  '2025-04-01': {'1. open': '76229.03000000',
   '2. high': '79232.59000000',
   '3. low': '76149.07000000',
   '4. close': '78902.29000000',
   '5. v

### transform json into comprehensive data to load to Railway

##### check data already loaded in PosgreSQL Table hosted in Railway

In [117]:
db_conn = os.getenv("DB_CONN")
conn = psycopg2.connect(db_conn)
cur = conn.cursor()

cur.execute('''
    SELECT MAX(date)FROM api_data;
                ''')
last_date_PostgreSQL = cur.fetchall()
cur.close()
conn.close()

# amend the format to be a single value and not a list of one tuple
last_date_PostgreSQL_2=last_date_PostgreSQL[0][0]
last_date_PostgreSQL_2


datetime.datetime(2025, 3, 31, 0, 0)

##### check data received from the API

In [110]:
# find first block where we have all data we need to extract
first_block = list(data.keys())[1]
first_block

'Time Series (Digital Currency Daily)'

In [89]:
# extract dates available in the last data received from the API
List_dates_values_formatted = [datetime.strptime(key, "%Y-%m-%d") for key in data[first_block]]
print(List_dates_values_formatted)

[datetime.datetime(2025, 4, 4, 0, 0), datetime.datetime(2025, 4, 3, 0, 0), datetime.datetime(2025, 4, 2, 0, 0), datetime.datetime(2025, 4, 1, 0, 0), datetime.datetime(2025, 3, 31, 0, 0), datetime.datetime(2025, 3, 30, 0, 0), datetime.datetime(2025, 3, 29, 0, 0), datetime.datetime(2025, 3, 28, 0, 0), datetime.datetime(2025, 3, 27, 0, 0), datetime.datetime(2025, 3, 26, 0, 0), datetime.datetime(2025, 3, 25, 0, 0), datetime.datetime(2025, 3, 24, 0, 0), datetime.datetime(2025, 3, 23, 0, 0), datetime.datetime(2025, 3, 22, 0, 0), datetime.datetime(2025, 3, 21, 0, 0), datetime.datetime(2025, 3, 20, 0, 0), datetime.datetime(2025, 3, 19, 0, 0), datetime.datetime(2025, 3, 18, 0, 0), datetime.datetime(2025, 3, 17, 0, 0), datetime.datetime(2025, 3, 16, 0, 0), datetime.datetime(2025, 3, 15, 0, 0), datetime.datetime(2025, 3, 14, 0, 0), datetime.datetime(2025, 3, 13, 0, 0), datetime.datetime(2025, 3, 12, 0, 0), datetime.datetime(2025, 3, 11, 0, 0), datetime.datetime(2025, 3, 10, 0, 0), datetime.dateti

##### prepare data to load in railway

In [111]:
# find dates received from the API that are not in the PosgreSQL table yet
dates_to_append = [x for x in List_dates_values_formatted if x > last_date_PostgreSQL_2]
print(dates_to_append)

[datetime.datetime(2025, 4, 4, 0, 0), datetime.datetime(2025, 4, 3, 0, 0), datetime.datetime(2025, 4, 2, 0, 0), datetime.datetime(2025, 4, 1, 0, 0)]


In [112]:
dates_to_append_format = [key.strftime("%Y-%m-%d") for key in dates_to_append]
print(dates_to_append_format)

['2025-04-04', '2025-04-03', '2025-04-02', '2025-04-01']


In [ ]:
N = len(dates_to_append_format)
a_ = dates_to_append_format[:N]
a = list(a_)

In [ ]:
data_rows_to_append=[]
for date_to_append in a:
    # return the five Bitcoin values
    bitcoin_values= [obj for d, obj in data[first_block].get(date_to_append).items()][:5]
    new_row = (
        date_to_append,
        float(bitcoin_values[0]),
        float(bitcoin_values[1]),
        float(bitcoin_values[2]),
        float(bitcoin_values[3]),
        float(bitcoin_values[4])
    )
    # print(new_row)
    data_rows_to_append.append(new_row)
data_rows_to_append


[('2025-04-04', 75324.46, 75535.33, 75075.93, 75099.15, 7.39032644),
 ('2025-04-03', 75661.51, 76895.06, 73069.0, 75302.11, 710.68541996),
 ('2025-04-02', 78900.68, 81300.0, 75600.0, 75617.59, 685.40827378),
 ('2025-04-01', 76229.03, 79232.59, 76149.07, 78902.29, 416.94996883)]

### add data to the table in railway

In [119]:
db_conn = os.getenv("DB_CONN")
conn = psycopg2.connect(db_conn)
cur = conn.cursor()

cur.executemany(
    '''
    INSERT INTO api_data(date, open, high, low, close, volume)
    VALUES (%s, %s, %s, %s, %s, %s);
    ''', data_rows_to_append)


In [120]:
conn.commit()

In [121]:
cur.close()
conn.close()